In [1]:
import numpy as np

In [2]:
CONST_ND = 1/(np.sqrt(np.power(2*np.pi, 13)))

def calculate_b(m, C, o):
    det = 1/np.sqrt((np.linalg.det(C)))
    return CONST_ND*det*np.exp(-0.5*(np.transpose((o - m))@np.linalg.inv(C)@(o-m)))



def recalculate_bo(C, m, data):
    b_ano = np.zeros((33, 5), dtype=float)


    for i in range(data.shape[0]):
        b_ano[i, 1] = calculate_b(
        m[0, :],
        C[0, :, :],
        data[i, :]
        )

        b_ano[i, 2] = calculate_b(
            m[1, :],
            C[1, :, :],
            data[i, :]
        )

        b_ano[i, 3] = calculate_b(
            m[2, :],
            C[2, :, :],
            data[i, :]
        )

    return b_ano




def recalculate_alpha(a, b_ano):
    alpha = np.zeros((33, 5))
    ANO = a


    alpha[0, 0] = ANO[0, 0]*b_ano[0, 0]
    alpha[0, 1] = ANO[0, 1]*b_ano[0, 1]
    alpha[0, 2] = ANO[0, 2]*b_ano[0, 2]
    alpha[0, 3] = ANO[0, 3]*b_ano[0, 3]
    alpha[0, 4] = ANO[0, 4]*b_ano[0, 4]

    for k in range(1, 33, 1):
        sum = 0
        for i in range(5):
            sum += alpha[k-1, i]*ANO[i, 0]

        alpha[k, 0] = sum*b_ano[k, 0]

        sum = 0
        for i in range(5):
            sum += alpha[k-1, i]*ANO[i, 1]

        alpha[k, 1] = sum*b_ano[k, 1]

        sum = 0
        for i in range(5):
            sum += alpha[k-1, i]*ANO[i, 2]

        alpha[k, 2] = sum*b_ano[k, 2]

        sum = 0
        for i in range(5):
            sum += alpha[k-1, i]*ANO[i, 3]

        alpha[k, 3] = sum*b_ano[k, 3]

        sum = 0
        for i in range(5):
            sum += alpha[k-1, i]*ANO[i, 4]

        alpha[k, 4] = sum*b_ano[k, 4]

    res_sum = 0
    for i in range(5):
        res_sum += alpha[32, i]*ANO[i, 4]

    print("result sum (forward): ", np.log(res_sum))

    return alpha, np.log(res_sum)


def recalculate_beta(a, b_ano):
    ANO = a
    beta = np.zeros((33, 5))

    for i in range(5):
        beta[32, i] = ANO[i, 4]




    # ---
    for k in range(31, -1, -1):
        for n in range(4, 0, -1):
            sum = 0
            for m in range(0, 5, 1):
                sum += ANO[n, m]*beta[k+1, m]*b_ano[k+1, m]
            beta[k, n] = sum

    #---

    sum = 0
    for i in range(5):
        sum += ANO[0, i]*b_ano[0, i]*beta[0, i]

    print("result sum (backward):", np.log(sum))

    return beta, np.log(sum)






In [ ]:
def calculate_new_mu(alpha, beta, o):
    mu = np.zeros((3, 13))
    sum_1_abo = 0
    for i in range(alpha.shape[0]):
        sum_1_abo += alpha[i, 1]*beta[i, 1]*o[i, :]
    sum_1_ab = 0
    for i in range(alpha.shape[0]):
        sum_1_ab += alpha[i, 1]*beta[i, 1]
    mu[0, :] = sum_1_abo/sum_1_ab

    sum_2_abo = 0
    for i in range(alpha.shape[0]):
        sum_2_abo += alpha[i, 2]*beta[i, 2]*o[i, :]
    sum_2_ab = 0
    for i in range(alpha.shape[0]):
        sum_2_ab += alpha[i, 2]*beta[i, 2]
    mu[1, :] = sum_2_abo/sum_2_ab

    sum_3_abo = 0
    for i in range(alpha.shape[0]):
        sum_3_abo += alpha[i, 3]*beta[i, 3]*o[i, :]
    sum_3_ab = 0
    for i in range(alpha.shape[0]):
        sum_3_ab += alpha[i, 3]*beta[i, 3]
    mu[2, :] = sum_3_abo/sum_3_ab

    return mu


def calculate_new_cov(mu, alpha, beta, o):

    covariance_new = np.zeros((3, 13, 13))

    sum_ab_1 = 0
    for i in range(alpha.shape[0]):
        sum_ab_1 += alpha[i, 1]*beta[i, 1]
    sum_cov_1 = 0
    for i in range(alpha.shape[0]):
        sum_cov_1 += alpha[i, 1]*beta[i, 1]*((o[i, :] - mu[0, :])*np.transpose(o[i, :] - mu[0, :]))
    covariance_new[0, :, :] = np.diag(sum_cov_1)/sum_ab_1


    sum_ab_2 = 0
    for i in range(alpha.shape[0]):
        sum_ab_2 += alpha[i, 2]*beta[i, 2]
    sum_cov_2 = 0
    for i in range(alpha.shape[0]):
        sum_cov_2 += alpha[i, 2]*beta[i, 2]*((o[i, :] - mu[1, :])*np.transpose(o[i, :] - mu[1, :]))
    covariance_new[1, :, :] = np.diag(sum_cov_2)/sum_ab_2

    sum_ab_3 = 0
    for i in range(alpha.shape[0]):
        sum_ab_3 += alpha[i, 3]*beta[i, 3]
    sum_cov_3 = 0
    for i in range(alpha.shape[0]):
        sum_cov_3 += alpha[i, 3]*beta[i, 3]*((o[i, :] - mu[2, :])*np.transpose(o[i, :] - mu[2, :]))
    covariance_new[2, :, :] = np.diag(sum_cov_3)/sum_ab_3

    return covariance_new



def calculate_new_a(alpha, a, beta, b_ano):
    T, N = alpha.shape
    a_new = np.zeros((N, N))

    for i in range(N):
        denom = np.sum(alpha[:, i] * beta[:, i])
        for j in range(N - 1):
            num = 0
            for t in range(T - 1):
                num += alpha[t, i] * a[i, j] * b_ano[t + 1, j] * beta[t + 1, j]
            a_new[i, j] = num / denom if denom != 0 else 0.0

    for i in range(N):
        denom = np.sum(alpha[:, i] * beta[:, i])
        num = alpha[-1, i] * beta[-1, i]
        a_new[i, N - 1] = num / denom if denom != 0 else 0.0

    a_new[0, 1] = 1

    return a_new
    
    


In [4]:
path_to_o = r"D:\prednasky\ARR\test_1_3.txt"
data = np.loadtxt(path_to_o)

print(data)

[[ 5.942524e+00 -3.282736e-01 -3.374459e+00 -1.136272e+00 -9.278540e-02
   7.558367e-01 -1.307173e+00 -5.358572e-01 -1.091770e+00 -1.017745e+00
  -3.463146e-01  1.505566e-01 -3.405006e-01]
 [ 6.730464e+00 -7.462602e-01 -3.328790e+00 -1.142270e+00  6.955092e-02
   4.836368e-01 -1.201545e+00 -8.939514e-01 -8.775608e-01 -1.212151e+00
  -2.820395e-01  2.394679e-01 -2.724751e-01]
 [ 6.320567e+00 -3.989284e-01 -3.527767e+00 -1.192769e+00 -2.196304e-02
   2.780739e-01 -9.985700e-01 -9.565379e-01 -9.987731e-01 -1.258727e+00
  -3.555308e-01  1.548847e-01 -3.194805e-01]
 [ 6.779319e+00 -2.770809e-01 -3.229155e+00 -1.284317e+00 -3.337887e-01
   2.896500e-01 -1.198773e+00 -8.141817e-01 -1.244111e+00 -1.149649e+00
  -1.856714e-01  2.255759e-01 -3.591545e-01]
 [ 5.780449e+00  3.891160e-02 -2.500877e+00 -1.640934e+00 -3.873832e-01
   3.870268e-02 -1.070985e+00 -4.445389e-01 -1.139093e+00 -1.246184e+00
  -2.819542e-02  4.406920e-02 -2.282665e-01]
 [ 6.023695e+00  1.223905e-01 -2.882414e+00 -1.312963e+

In [ ]:
#inicializace
mean = np.zeros((1, data.shape[1]))
a = np.zeros((5, 5))
cov = np.zeros((13, 13))

a[0, 1] = 1
a[1, 1] = 0.5
a[1, 2] = 0.5
a[2, 2] = 0.5
a[2, 3] = 0.5
a[3, 3] = 0.5
a[3, 4] = 0.5


for i in range(data.shape[1]):
    mean[0, i] = sum(data[:, i])/(data.shape[0])


variances = np.var(data, axis=0, ddof=1)
covariance = np.diag(variances)

mean_res = np.zeros((3, data.shape[1]))
covariance_res = np.zeros((3, 13, 13))

for i in range(3):
    mean_res[i, :] = mean[0, :]

for i in range(3):
    covariance_res[i, :, :] = covariance[:, :]



b_ano = recalculate_bo(covariance_res, mean_res, data)

alpha, _ = recalculate_alpha(a, b_ano=b_ano)
beta, _ = recalculate_beta(a, b_ano)

mu = calculate_new_mu(alpha, beta, data)
cov = calculate_new_cov(mu, alpha, beta, data)

a = calculate_new_a(alpha, a, beta, b_ano)


for _ in range(4):
    b_ano = recalculate_bo(cov, mu, data)
    alpha, _ = recalculate_alpha(a, b_ano)
    beta, _ = recalculate_beta(a, b_ano)
    mu = calculate_new_mu(alpha, beta, data)
    cov = calculate_new_cov(mu, alpha, beta, data)
    a = calculate_new_a(alpha, a, beta, b_ano)
    



result sum (forward):  -361.7244566386628
result sum (backward): -361.7244566386628
result sum (forward):  -228.28541382812688
result sum (backward): -228.28541382812688
result sum (forward):  -139.52117551473512
result sum (backward): -139.52117551473512
result sum (forward):  -126.91627172708154
result sum (backward): -126.91627172708154
result sum (forward):  -119.65412635770774
result sum (backward): -119.65412635770774
